# 10. 스태킹 앙상블

<br>

# 1. 스태킹(Stacking)

## 1.1 스태킹(Stacking)

- 개별적인 여러 알고리즘을 서로 결합해 예측 결과를 도출 (배깅 및 부스팅과의 공통점)
- 개별 알고리즘으로 예측한 **데이터를 기반**으로 다시 예측을 수행 (배깅 및 부스팅과의 차이점)

<br>

## 1.2 메타 모델 방식

- 개별 모델의 예측된 데이터 세트를 다시 기반으로 학습하고 예측하는 방식  
  
  
- 개별 알고리즘의 예측 결과 데이터 세트를 최종적인 메타 데이터 세트로 만듬
- 이를 별도의 ML 알고리즘으로 최종 학습을 수행
- 테스트 데이터를 기반으로 다시 예측을 수행하는 방식

<br>

## 1.3 스태킹 모델의 종류

- 스태킹 모델은 두 종류의 모델이 필요  
  
  
1. **개별적인 기반 모델**
2. **최종 메타 모델** : 개별 기반 모델의 예측 데이터를 학습 데이터로 만들어서 학습  

<br>

## 1.4 스태킹 모델의 핵심

- 여러 개별 모델의 예측 데이터를 각각 스태킹 형태로 결합
- 최종 메타 모델의 학습용 피처 데이터 세트와 테스트용 피처 데이터 세트로 만드는 것

<br>

## 1.5 스태킹의 사용

- 스태킹을 현실 모델에 적용하는 경우는 많지 않음
- 스태킹을 적용할 때 많은 개별 모델이 필요
- 2~3개의 개별 모델만을 결합해서는 쉽게 예측 성능을 향상시킬 수 없음
- 일반적으로 성능이 비슷한 모델을 결합해 좀 더 나은 성능 향상을 도출하기 위해 적용

<br>

## 1.6 스태킹 모델 개념

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img001.jpg' width='300px'/>
</div>

- 여러 개의 모델에 대한 예측값을 합함
- 즉 스태킹 형태로 쌓은 뒤 이에 대한 예측을 다시 수행

<br>

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img002.jpg' width='700px'/>
</div>

- M개의 로우, N개의 피처(컬럼)을 가진 데이터 세트에 스태킹 앙상블을 적용한다고 가정

- 학습에 사용할 ML 알고리즘 모델 : 3개
- 먼저 모델별로 각각 학습을 시킨 뒤 예측을 수행 $\rightarrow$ M개의 로우를 가진 1개의 레이블 값을 도출
- 모델별로 도출된 예측 레이블 값을 다시 합해서(스태킹) 새로운 데이터 세트를 만듬
- 이렇게 스태킹된 데이터 세트에 대해 최종 모델을 적용해 최종 예측을 실시

<br>

# 2. 기본 스태킹 모델

- 위스콘신 암 데이터 세트에 기본 스태킹 모델을 적용

In [2]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

<br>

스태킹에 사용될 개별 모델

- KNN
- 랜덤 포레스트
- 결정 트리
- 에이다부스트  

스태킹에 사용될 최종 모델

- 로지스틱 회귀

In [4]:
# 개별 ML 모델 생성

knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 스태킹으로 만들어진 데이터 세트를 학습, 예측할 최종 모델
lr_final = LogisticRegression(C=10)

<br>

개별 모델 학습

In [5]:
# 개별 모델들을 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

<br>

개별 모델의 예측 데이터 세트를 반환하고 각 모델의 예측 정확도 확인

In [6]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고 개별 모델의 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도 : {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도 : 0.9211
랜덤 포레스트 정확도 : 0.9649
결정 트리 정확도 : 0.9035
에이다부스트 정확도 : 0.9561


<br>

개별 알고리즘으로부터 예측된 예측값을 컬럼 레벨로 옆으로 붙여서 피처 값으로 만듬  
이를 최종 메타 모델인 로지스틱 회귀에서 학습 데이터로 다시 사용  
  
  
반환된 예측 데이터 세트는 1차원 형태의 ndarray이다.  
$\rightarrow$ 먼저 반환된 예측 결과를 행 형태로 붙임  
$\rightarrow$ 그런 다음 넘파이의 `transpose()`를 이용해 행과 열 위치를 바꾼 ndarray로 변환

In [7]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


In [8]:
# transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듬
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


<br>

예측 데이터로 생성된 데이터 세트를 기반으로 최종 메타 모델인 로지스틱 회귀를 학습 및 예측 정확도 측정

In [9]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, final)))

최종 메타 모델의 예측 정확도 : 0.9737


C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


$\rightarrow$ 4개의 개별 모델 중 가장 정확도가 높았던 랜덤 포레스트의 0.9649 보다 개선된 정확도를 얻음

<br>

# 3. CV 세트 기반의 스태킹

## 3.1 CV 세트 기반의 스태킹 모델

- 과적합을 개선하기 위해 최종 메타 모델을 위한 데이터 세트를 만들 때 교차 검증 기반으로 예측된 결과 데이터 세트를 이용  
  
  
- 앞 예제에서 마지막에 메타 모델인 로지스틱 회귀 모델 기반에서 최종 학습 시 테스트용 레이블 데이터 세트를 기반으로 학습  
$\rightarrow$ 과적합 문제가 발생할 수 있다.  
  
  
- CV 세트 기반의 스태킹은 이에 대한 개선
- 개별 모델들이 각각 교차 검증으로 메타 모델을 위한 학습용 스태킹 데이터 생성
- 예측을 위한 테스트용 스태킹 데이터 생성
- 이를 기반으로 메타 모델이 학습과 예측을 수행

**스텝 1**

- 각 모델별로 원본 학습/테스트 데이터를 예측한 결과 값을 기반으로 메타 모델을 위한 학습용/테스트용 데이터 생성  
  
  
**스텝 2**

- 스텝 1에서 개별 모델들이 생성한 학습용 데이터를 모두 스태킹 형태로 합침
- 이를 메타 모델이 학습할 최종 학습용 데이터 세트 생성  
  
  
- 마찬가지로 각 모델들이 생성한 테스트용 데이터를 모두 스태킹 형태로 합침
- 이를 메타 모델이 예측할 최종 테스트 데이터 세트를 생성  
  
  
- 메타 모델은 최종적으로 생성된 학습 데이터 세트와 원본 학습 데이터의 레이블 데이터를 기반으로 학습
- 최종적으로 생성된 테스트 데이터 세트를 예측
- 원본 테스트 데이터의 레비을 데이터를 기반으로 예측

**핵심**

- 개별 모델에서 메타 모델인 2차 모델에서 사용될 학습용 데이터와 테스트용 데이터를 교차 검증을 통해 생성하는 것
- 스텝 1은 개별 모델 레벨에서 수행
- 이러한 로직을 여러 개의 개별 모델에서 동일하게 수행

<br>

## 3.2 주요 프로세스

- 먼저 학습용 데이터를 N개의 폴드(Fold)로 나눔 (3개의 폴드세트로 가정)
- 3번의 유사한 반복 작업을 수행
- 마지막 3번째 반복에서 개별 모델의 예측값으로 학습 데이터와 테스트 데이터를 생성

### 3.2.1 데이터 폴드 나누기

- 학습용 데이터를 3개의 폴드로 나눔
- 이때 2개의 폴드는 학습을 위한 데이터 폴드
- 나머지 1개의 폴드는 검증을 위한 데이터 폴드로 나눔
- 이렇게 두 개의 폴드로 나뉜 학습 데이터를 기반으로 개별 모델을 학습시킴

### 3.2.2 메타 모델 학습을 위한 학습 데이터 생성

- 이렇게 학습된 개별 모델은 검증 폴드 1개 데이터로 예측
- 그 결과를 저장  
  
  
- 이러한 로직을 3번 반복하면서 학습 데이터와 검증 데이터 세트를 변경해가면서 학습 후 예측 결과를 별도로 저장
- 이렇게 만들어진 예측 데이터는 메타 모델을 학습시키는 학습 데이터로 사용

### 3.2.3 메타 모델 평가를 위한 테스트 데이터 생성

- 2개의 학습 폴드 데이터로 학습된 개별 모델은 원본 테스트 데이터를 예측하여 예측값 생성  
  
  
- 마찬가지로 이러한 로직을 3번 반복하면서 이 예측값의 평균으로 최종 결괏값을 생성
- 이를 메타 모델을 위한 테스트 데이터로 사용

<br>

## 3.3 상세 설명 : 스텝 1

- 스텝 1에서 수행되는 내용을 그림으로 설명

<br>

### 3.3.1 스태킹 데이터 생성 : 첫 번째 반복

- 3개의 폴드만큼 반복을 수행하면서 스태킹 데이터를 생성하는 첫 번째 반복을 설명

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img003.jpg' width='600px'/>
</div>

<br>

### 3.3.2 스태킹 데이터 생성 : 두 번째 반복

- 폴드 내의 학습용 데이터 세트를 변경
- 첫 번째 그림과 동일한 작업을 수행

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img004.jpg' width='600px'/>
</div>

<br>

### 3.3.3 스태킹 데이터 생성 : 세 번째 반복

- 세 번째 반복을 수행하면서 폴드 내의 학습용 데이터 데이터 세트가 변경됨  
  
  
- 세 번째 반복을 완료하면 첫 번째, 두 번째, 세 번째 반복을 수행하면서 만들어진 폴드별 예측 데이터를 합하여 메타 모델에서 사용될 학습 데이터를 만듬
- 마찬가지로 첫 번째, 두 번째, 세 번째 반복을 수행하면서 학습 폴드 데이터로 학습된 개별 모델이 원본 테스트 세트로 예측한 결괏값을 최종 평균하여 메타 모델에서 사용될 테스트 데이터를 만듬

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img005.jpg' width='600px'/>
</div>

<br>

## 3.4 상세 설명 : 스텝 2

- 각 모델들이 스텝 1로 생성한 학습과 테스트 데이터를 모두 합쳐서 최종적으로 메타 모델이 사용할 학습 데이터 테스트 데이터를 생성
- 메타 모델이 사용할 최종 학습 데이터와 원본 데이터의 레이블 데이터를 합쳐서 메타 모델을 학습
- 최종 테스트 데이터로 예측을 수행
- 최종 예측 결과를 원본 테스트 데이터의 레이블 데이터와 비교 평가

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch04/10/img006.jpg' width='600px'/>
</div>

<br>

## 3.5 스텝 1 코드 구현

`get_stacking_base_datasets()`  

- 합수 입력 값
  - 개별 모델의 Classifier 객체
  - 원본인 학습용 피처 데이터
  - 원본인 학습용 레이블 데이터
  - 원본인 테스트 피처 데이터
  - K 폴드를 몇 개로 할 지를 지정하는 파라미터  
  
  
- 폴드의 개수만큼 반복을 수행
- 폴드된 학습용 데이터로 학습
- 예측 결괏값을 기반으로 메타 모델을 위한 학습용 데이터와 테스트용 데이터를 새롭게 생성

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds) :
    
    # 지정된 n_folds값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    
    # 추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, ' model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)) :
        
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트 : ', folder_counter, ' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        
        # 입력된 원본 테스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    # train_fold_pred : 최종 메타 모델이 사용하는 학습 데이터
    # test_pred_mean : 최종 메타 모델이 사용하는 테스트 데이터
    return train_fold_pred, test_pred_mean

<br>

개별 모델들(KNN, 랜덤 포레스트, 결정 트리, 에이다부스트)에 `get_stacking_base_datasets()` 함수를 호출해 각각 메타 모델이 추후에 사용할 학습용, 테스트용 데이터 세트 반환

In [12]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
RandomForestClassifier  model 시작
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
DecisionTreeClassifier  model 시작
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
AdaBoostClassifier  model 시작
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 


<br>

## 3.6 스텝 2 코드 구현

- `get_stacking_base_datasets()` 호출로 반환된 각 모델별 학습 데이터와 테스트 데이터를 결합
- 넘파이의 `concatenate()`를 이용해 이와 같은 기능을 수행
- `concatenate()`는 여러 개의 넘파이 배열을 컬럼 또는 로우 레벨로 합쳐주는 기능 제공

In [14]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)

print('원본 학습 피처 데이터 Shape : ', X_train.shape, ' 원본 테스트 피처 데이터 Shape : ', X_test.shape)
print('스태킹 학습 피처 데이터 Shape : ', Stack_final_X_train.shape, ' 스태킹 테스트 피처 데이터 Shape : ', Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape :  (455, 30)  원본 테스트 피처 데이터 Shape :  (114, 30)
스태킹 학습 피처 데이터 Shape :  (455, 4)  스태킹 테스트 피처 데이터 Shape :  (114, 4)


$\rightarrow$ `Stack_final_X_train` : 메타 모델이 학습할 학습용 피처 데이터 세트  
$\rightarrow$ `Stack_final_X_test` : 메타 모델이 예측할 테스트용 피처 데이터 세트  
  
  
$\rightarrow$ 스태킹 학습 피처 데이터는 원본 학습 피처 데이터와 로우(Raw)크기는 같음  
$\rightarrow$ 4개의 개별 모델 예측값을 합쳤기 때문에 컬럼(Column) 크기는 4이다.

<br>

최종 메타 모델인 로지스틱 회귀를 스태킹된 학습용 피처 데이터 세트와 원본 학습 레이블 데이터로 학습  
그런 다음 스태킹된 테스트 데이터로 예측  
예측 결과를 원본 테스트 레이블 데이터와 비교해 정확도 측정

In [16]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델ㄹ의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델ㄹ의 예측 정확도 : 0.9737


C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<br>

# 4. 스태킹 파라미터 튜닝

- 스태킹을 이루는 모델은 최적으로 파라미터를 튜닝한 상태에서 스태킹 모델을 만드는 것이 일반적
- 여러 명으로 이뤄진 분석 팀에서 개별적으로 각각 모델을 최적으로 학습시켜 스태킹 모델을 더 빠르게 최적화할 수 있다.
- 스태킹 모델의 파라미터 튜닝 : 개별 알고리즘 모델의 파라미터를 최적으로 튜닝하는 것을 의미